In [ ]:
%load_ext autoreload
%autoreload 2
from pythonLib.pixelationLib import*
from pythonLib.formImageLib import*

input_file_path = "../ADS6311/positive/histogramData/positive_histogram_1.h5"
pixel_hist = display_image_fromH5(input_file_path)
pixel_hist = pixel_hist[15, 25, :]  # shape: (bin_num,)
bin_centers = np.arange(len(pixel_hist))  # or use bin_width and range_min if needed
plt.figure()
plt.bar( bin_centers, pixel_hist, width=1)
plt.xlabel("Bin Index")
plt.ylabel("Count")
plt.grid(True)
plt.show()

In [ ]:
%load_ext autoreload
%autoreload 2
from pythonLib.pixelationLib import*
from pythonLib.formImageLib import*

# input_file_path = "../ADS6311/positive/rawData/positive_rawData_1.h5"
input_file_path = "../ADS_calibration/rawData/static_rawData_4.h5"
emission_delay = read_emission_delay(input_file_path)

if emission_delay.size > 0:
    bin_width = 114
    range_min = np.min(emission_delay)
    range_max = np.max(emission_delay)
    bins = np.arange(range_min, range_max+ bin_width, bin_width)

    # Number of bins = (range span / bin_width)
    print(emission_delay)
    plt.figure(figsize=(8, 5))
    plt.hist(emission_delay, bins=bins, density=False,  # density=False = raw counts
             alpha=0.7, color='steelblue', edgecolor='black')
    plt.title("Emission Delay Distribution")
    plt.xlabel("Delay (seconds)")
    plt.ylabel("Probability Density")
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
%load_ext autoreload
%autoreload 2
from pythonLib.pixelationLib import*
from pythonLib.formImageLib import*
from scipy.stats import norm
from scipy.optimize import curve_fit

calibration_input_file_1 = "../ADS_calibration/pixelizedData/static_pixelized_3.h5"
calibration_input_file_2 = "../ADS_calibration/pixelizedData/static_pixelized_4.h5"
calibration_input_file_3 = "../ADS_calibration/pixelizedData/static_pixelized_5.h5"
calibration_input_file_4 = "../ADS_calibration/pixelizedData/static_pixelized_6.h5"

range_min = 1480
bin_number = 600 
bin_width = 22.5 
range_max = range_min + bin_number * bin_width
myRange = [range_min,range_max]

pixels_1, image_width, image_heigh = decode_file(calibration_input_file_1)
pixels_2, image_width, image_heigh = decode_file(calibration_input_file_2)
pixels_3, image_width, image_heigh = decode_file(calibration_input_file_3)
pixels_4, image_width, image_heigh = decode_file(calibration_input_file_4)


# pixels_1,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_1,imageWidth=image_width,imageHeight=image_heigh, pluse_train_num = 1500)
# pixels_2,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_2,imageWidth=image_width,imageHeight=image_heigh, pluse_train_num = 1500)
# pixels_3,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_3,imageWidth=image_width,imageHeight=image_heigh, pluse_train_num = 1500)
# pixels_4,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_4,imageWidth=image_width,imageHeight=image_heigh, pluse_train_num = 1500)

image_1, illegal_photon, stamped_histogram_1, stamped_collosion = form_histogram_image(pixels_1,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image_2, illegal_photon, stamped_histogram_2, stamped_collosion = form_histogram_image(pixels_2,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image_3, illegal_photon, stamped_histogram_3, stamped_collosion = form_histogram_image(pixels_3,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image_4, illegal_photon, stamped_histogram_4, stamped_collosion = form_histogram_image(pixels_4,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)

display_image(image_1,distance_range=myRange)
histogram_120=display_pixel_histogram_from_matrix(stamped_histogram_1,25,25,display=True)
display_image(image_2,distance_range=myRange)
histogram_240=display_pixel_histogram_from_matrix(stamped_histogram_2,25,25)
display_image(image_3,distance_range=myRange)
histogram_360=display_pixel_histogram_from_matrix(stamped_histogram_3,25,25)
display_image(image_4,distance_range=myRange)
histogram_480=display_pixel_histogram_from_matrix(stamped_histogram_4,25,25)

# --- Define quadratic function ---
def quad_func(x, a, c):
    return c - a / x**2  

def linear_func(x,a,b):
    return a - b*x

def estimate_normal_from_hist(pixel_hist, bin_width=1.0, range_min=0.0):
    """
    Estimate mean and variance of a normal distribution
    from a histogram (pixel_hist).
    
    Args:
        pixel_hist: 1D array of histogram counts
        bin_width:  width of each bin (e.g., in ps or mm)
        range_min:  start position of first bin

    Returns:
        mean, variance
    """
    total = np.sum(pixel_hist)
    if total == 0:
        return np.nan, np.nan  # avoid division by zero

    # Compute bin centers
    bin_centers = np.arange(len(pixel_hist)) * bin_width + range_min

    # Weighted mean and variance
    mean = np.sum(bin_centers * pixel_hist) / total
    variance = np.sum(pixel_hist * (bin_centers - mean)**2) / total

    return mean, variance




# Fit 120 cm
mean_120, var_120 = estimate_normal_from_hist(histogram_120)
sigma_120 = np.sqrt(var_120)
x = np.linspace(0, len(histogram_120)-1, 600)
pdf_120 = norm.pdf(x, mean_120, sigma_120)
pdf_120_scaled = pdf_120 * np.max(histogram_120) / np.max(pdf_120)

# Fit 240 cm
mean_240, var_240 = estimate_normal_from_hist(histogram_240)
sigma_240 = np.sqrt(var_240)
pdf_240 = norm.pdf(x, mean_240, sigma_240)
pdf_240_scaled = pdf_240 * np.max(histogram_240) / np.max(pdf_240)

# Fit 360 cm
mean_360, var_360 = estimate_normal_from_hist(histogram_360)
sigma_360 = np.sqrt(var_360)
pdf_360 = norm.pdf(x, mean_360, sigma_360)
pdf_360_scaled = pdf_360 * np.max(histogram_360) / np.max(pdf_360)

# Fit 480 cm
mean_480, var_480 = estimate_normal_from_hist(histogram_480)
sigma_480 = np.sqrt(var_480)
pdf_480 = norm.pdf(x, mean_480, sigma_480)
pdf_480_scaled = pdf_480 * np.max(histogram_480) / np.max(pdf_480)

# Store results
x_data = np.array([(mean_120), (mean_240), (mean_360), (mean_480)])  # distances (cm)
y_data = np.array([np.max(histogram_120),np.max(histogram_240),np.max(histogram_360),np.max(histogram_480) ])  # mean bin

# --- Fit curve ---
popt, pcov = curve_fit(quad_func, x_data, y_data)
a, c = popt
print(f"Fitted coefficients:\n a = {a:.6e}\n c = {c:.6e}")

# y_data =  y_data + c 
# popt, pcov = curve_fit(quad_func, x_data, y_data)
# a, c = popt


# --- Generate smooth fitted curve for plotting ---
x_fit = np.linspace(42, 600, 600)
y_fit = quad_func(x_fit, a, c)

bin_centers = np.arange(len(histogram_120))  # or use bin_width and range_min if needed
plt.figure()
plt.bar(bin_centers,histogram_120, width=1, alpha=0.6, color="blue", label="120cm")
plt.bar(bin_centers,histogram_240, width=1, alpha=0.6, color="red", label="240cm")
plt.bar(bin_centers,histogram_360, width=1, alpha=0.6, color="green", label="360cm")
plt.bar(bin_centers,histogram_480, width=1, alpha=0.6, color="orange", label="480cm")
plt.xlabel("Bin")
plt.ylabel("Count")

plt.plot(x, pdf_120_scaled, "k--", linewidth=1,
         label=f"Gaussian fit 120 (μ={(range_min+mean_120*bin_width)/20:.1f}, σ={sigma_120:.1f})")

plt.plot(x, pdf_240_scaled, "k--", linewidth=1,
         label=f"Gaussian fit 240 (μ={(range_min+mean_240*bin_width)/20:.1f}, σ={sigma_240:.1f})")


plt.plot(x, pdf_360_scaled, "k--", linewidth=1,
         label=f"Gaussian fit 360 (μ={(range_min+mean_360*bin_width)/20:.1f}, σ={sigma_360:.1f})")

plt.plot(x, pdf_480_scaled, "k--", linewidth=1,
         label=f"Gaussian fit 480 (μ={(range_min+mean_480*bin_width)/20:.1f}, σ={sigma_480:.1f})")



plt.plot(x_data, y_data, 'o',  color='red', markersize=3)
plt.plot(x_fit, y_fit, 'r--', linewidth=1)

# plt.yscale('log')   
# plt.ylim(1, 600)

# plt.xscale('log')
# plt.xlim(10,600)
plt.grid()
plt.legend()
plt.savefig("wall_experiment.png", dpi=300, bbox_inches='tight', transparent=True)
plt.show()




In [ ]:
def filter_photon(simulated_photon, min_time=0, max_time=np.inf):
    """
    Traverse rows and columns of a 2D array of photon lists.
    Keep only photons with arrival time between min_time and max_time.
    """
    height = len(simulated_photon)
    width = len(simulated_photon[0])

    output = [[[] for _ in range(width)] for _ in range(height)]
    process_count = 0
    pre_count = 0
    for y in range(height):
        for x in range(width):
            entries = simulated_photon[y][x]
            pre_count = pre_count + len(entries) 
            filtered = [e for e in entries if  e[1] == 2]
            output[y][x] = filtered
            process_count= process_count+ len(filtered)
    print(f'start count {pre_count}')
    print(f'filter count {process_count}')
    return output  






pixels_1 = filter_photon(pixels_1)
pixels_2 = filter_photon(pixels_2)
pixels_3 = filter_photon(pixels_3)
pixels_4 = filter_photon(pixels_4)





image_1, illegal_photon, stamped_histogram_1, stamped_collosion = form_histogram_image(pixels_1,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image_2, illegal_photon, stamped_histogram_2, stamped_collosion = form_histogram_image(pixels_2,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image_3, illegal_photon, stamped_histogram_3, stamped_collosion = form_histogram_image(pixels_3,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image_4, illegal_photon, stamped_histogram_4, stamped_collosion = form_histogram_image(pixels_4,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)

display_image(image_1,distance_range=myRange)
histogram_120=display_pixel_histogram_from_matrix(stamped_histogram_1,25,25)
# display_image(image_2,distance_range=myRange)
histogram_240=display_pixel_histogram_from_matrix(stamped_histogram_2,25,25)
# display_image(image_3,distance_range=myRange)
histogram_360=display_pixel_histogram_from_matrix(stamped_histogram_3,25,25)
# display_image(image_4,distance_range=myRange)
histogram_480=display_pixel_histogram_from_matrix(stamped_histogram_4,25,25)

# Fit 120 cm
mean_120, var_120 = estimate_normal_from_hist(histogram_120)
sigma_120 = np.sqrt(var_120)
x = np.linspace(0, len(histogram_120)-1, 600)
pdf_120 = norm.pdf(x, mean_120, sigma_120)
pdf_120_scaled = pdf_120 * np.max(histogram_120) / np.max(pdf_120)

# Fit 240 cm
mean_240, var_240 = estimate_normal_from_hist(histogram_240)
sigma_240 = np.sqrt(var_240)
pdf_240 = norm.pdf(x, mean_240, sigma_240)
pdf_240_scaled = pdf_240 * np.max(histogram_240) / np.max(pdf_240)

# Fit 360 cm
mean_360, var_360 = estimate_normal_from_hist(histogram_360)
sigma_360 = np.sqrt(var_360)
pdf_360 = norm.pdf(x, mean_360, sigma_360)
pdf_360_scaled = pdf_360 * np.max(histogram_360) / np.max(pdf_360)

# Fit 480 cm
mean_480, var_480 = estimate_normal_from_hist(histogram_480)
sigma_480 = np.sqrt(var_480)
pdf_480 = norm.pdf(x, mean_480, sigma_480)
pdf_480_scaled = pdf_480 * np.max(histogram_480) / np.max(pdf_480)

# Store results
x_data = np.array([(mean_120), (mean_240), (mean_360), (mean_480)])  # distances (cm)
y_data = np.array([np.max(histogram_120),np.max(histogram_240),np.max(histogram_360),np.max(histogram_480) ])  # mean bin

# --- Fit curve ---
popt, pcov = curve_fit(quad_func, x_data, y_data)
a, c = popt
print(f"Fitted coefficients:\n a = {a:.6e}\n c = {c:.6e}")

# --- Generate smooth fitted curve for plotting ---
x_fit = np.linspace(42, 600, 600)
y_fit = quad_func(x_fit, a, c)

bin_centers = np.arange(len(histogram_120))  # or use bin_width and range_min if needed
plt.figure()
plt.bar(bin_centers,histogram_120, width=1, alpha=0.6, color="blue", label="120cm")
plt.bar(bin_centers,histogram_240, width=1, alpha=0.6, color="red", label="240cm")
plt.bar(bin_centers,histogram_360, width=1, alpha=0.6, color="green", label="360cm")
plt.bar(bin_centers,histogram_480, width=1, alpha=0.6, color="orange", label="480cm")
plt.xlabel("Bin")
plt.ylabel("Count")

plt.plot(x, pdf_120_scaled, "k--", linewidth=1,
         label=f"Gaussian fit (μ={(range_min+mean_120*bin_width)/20:.1f}, σ={sigma_120:.1f})")

plt.plot(x, pdf_240_scaled, "k--", linewidth=1,
         label=f"Gaussian fit (μ={(range_min+mean_240*bin_width)/20:.1f}, σ={sigma_240:.1f})")


plt.plot(x, pdf_360_scaled, "k--", linewidth=1,
         label=f"Gaussian fit (μ={(range_min+mean_360*bin_width)/20:.1f}, σ={sigma_360:.1f})")

plt.plot(x, pdf_480_scaled, "k--", linewidth=1,
         label=f"Gaussian fit (μ={(range_min+mean_480*bin_width)/20:.1f}, σ={sigma_480:.1f})")

plt.plot(x_data, y_data, 'o',  color='red', markersize=3)
plt.plot(x_fit, y_fit, 'r--', linewidth=1)

plt.grid()
plt.legend()
plt.show()
# outputFile_1 = "../dead_time_simulation/dead_time_histogram_120.h5"
# outputFile_2 = "../dead_time_simulation/dead_time_histogram_240.h5"
# outputFile_3 = "../dead_time_simulation/dead_time_histogram_360.h5"
# outputFile_4 = "../dead_time_simulation/dead_time_histogram_480.h5"
# save_histogram_to_h5(outputFile_1, stamped_histogram_1, stamped_collosion ,range_min, range_max, image_width, image_heigh, bin_number)
# save_histogram_to_h5(outputFile_2, stamped_histogram_2, stamped_collosion ,range_min, range_max, image_width, image_heigh, bin_number)
# save_histogram_to_h5(outputFile_3, stamped_histogram_3, stamped_collosion ,range_min, range_max, image_width, image_heigh, bin_number)
# save_histogram_to_h5(outputFile_4, stamped_histogram_4, stamped_collosion ,range_min, range_max, image_width, image_heigh, bin_number)

In [ ]:
%load_ext autoreload
%autoreload 2
from pythonLib.pixelationLib import*
from pythonLib.formImageLib import*
from scipy.stats import norm
from scipy.optimize import curve_fit



calibration_input_file_1 = "../ADS_calibration/pixelizedData/static_pixelized_3.h5"

range_min = 1480
bin_number = 80 
bin_width = 22.5
range_max = range_min + bin_number * bin_width
myRange = [range_min,range_max]

pixels_1, image_width, image_heigh = decode_file(calibration_input_file_1)

pixels_1 = filter_photon(pixels_1)

def estimate_normal(photon_information):
    photon = [float(p[0]) for p in photon_information]
    return np.mean(photon),np.std(photon)


m,s = estimate_normal(pixels_1[25][25])

print("mean" + str(m) + " std " + str(s))
image_1, illegal_photon, stamped_histogram_1, stamped_collosion = form_histogram_image(pixels_1,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)

new_pixels,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_1,imageWidth=image_width,imageHeight=image_heigh,pluse_train_num = 20000)

image_1, illegal_photon, stamped_histogram_1, stamped_collosion = form_histogram_image(pixels_1,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)

dead_time_image_120, dead_time_illegal_photon_120, dead_time_stamped_histogram_120, dead_time_stamped_collosion_120 = form_histogram_image(new_pixels,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)

histogram_120=display_pixel_histogram_from_matrix(stamped_histogram_1,25,25)
dead_time_histogram_120 = display_pixel_histogram_from_matrix(dead_time_stamped_histogram_120,25,25)
print(sum(histogram_120))

# Fit 120 cm
mean_120, var_120 = estimate_normal_from_hist(histogram_120)
sigma_120 = np.sqrt(var_120)
x = np.linspace(0, len(histogram_120)-1, 600)
pdf_120 = norm.pdf(x, mean_120, sigma_120)
pdf_120_scaled = pdf_120 * np.max(histogram_120) / np.max(pdf_120)

# Fit the dead time 

dead_time_mean_120, dead_time_var_120 = estimate_normal_from_hist(dead_time_histogram_120)
dead_time_sigma_120 = np.sqrt(dead_time_var_120)
dead_time_pdf_120 = norm.pdf(x,dead_time_mean_120,dead_time_sigma_120)
dead_time_pdf_120_scaled = dead_time_pdf_120 * np.max(dead_time_histogram_120) / np.max(dead_time_pdf_120)

bin_centers = np.arange(len(histogram_120))  # or use bin_width and range_min if needed
plt.figure()
plt.grid(linestyle="--", alpha=0.3)
plt.gca().set_axisbelow(True)
plt.bar(bin_centers,histogram_120, width=1, alpha=0.6, color="blue", label="120cm",edgecolor="black")

plt.bar(bin_centers,dead_time_histogram_120, width=1, alpha=0.6, color="red", label="dead_time_120cm",edgecolor="black")


plt.plot(x, pdf_120_scaled, "k--", linewidth=1,
         label=f"Gaussian fit (μ={(range_min+mean_120*bin_width)/20:.1f}, σ={sigma_120:.1f})")

plt.plot(x, dead_time_pdf_120_scaled, "r--", linewidth=1,
         label=f"dead_time Gaussian fit (μ={(range_min+dead_time_mean_120*bin_width)/20:.1f}, σ={dead_time_sigma_120:.1f})")

plt.xlabel("Bin")
plt.ylabel("Count")

# plt.legend()
# plt.legend(loc='center left', bbox_to_anchor=(1.02, 0.5), borderaxespad=0.)
plt.legend(
    loc='center left', bbox_to_anchor=(1.005, 0.5),  # closer than 1.02
    borderaxespad=0.1,   # space between axes and legend
    borderpad=0.2,       # padding inside legend box
    labelspacing=0.2,    # vertical space between entries
    handlelength=1.0,    # length of line handles
    handletextpad=0.4,   # space between handle and text
    columnspacing=0.6,   # if using ncol>1
    fontsize=9,          # smaller text
    frameon=False        # drop the frame (often feels smaller)
)
plt.show()


In [ ]:
%load_ext autoreload
%autoreload 2
from pythonLib.pixelationLib import*
from pythonLib.formImageLib import*
from scipy.stats import norm
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import numpy as np



range_min = 1480
bin_number = 80 
bin_width = 22.5
range_max = range_min + bin_number * bin_width
myRange = [range_min,range_max]

mu, sigma = 2436,178 
rng = np.random.default_rng(48)
sample_number = 10000
samples = rng.normal(mu, sigma, sample_number)

pairs = [(float(p), 2) for p in samples]
input_sample = [[pairs]]

image_1, illegal_photon, stamped_histogram_1, stamped_collosion = form_histogram_image(input_sample,1,1,bin_number=bin_number,range_distance=myRange)
input_histogram = stamped_histogram_1[0][0]


dead_time_samples,discard_count,accept_count = dead_time_simulation(simulated_photon=input_sample,imageWidth=1,imageHeight=1,pluse_train_num = 20000)
image_1, illegal_photon, stamped_histogram_1, stamped_collosion = form_histogram_image(dead_time_samples,1,1,bin_number=bin_number,range_distance=myRange)

bin_centers = np.arange(len(input_histogram))  # or use bin_width and range_min if needed
plt.figure()
plt.grid(linestyle="--", alpha=0.3)
plt.gca().set_axisbelow(True)
plt.bar(bin_centers,input_histogram, width=1, alpha=0.6, color="blue", label="120cm",edgecolor="black")
plt.bar(bin_centers,stamped_histogram_1[0][0], width=1, alpha=0.6, color="red", label="120cm",edgecolor="black")
plt.show()


def dead_time_analysis(normal_mean,normal_sigma,pluse_train_num,photon_num_array):


    range_min = 1480
    bin_number = 80 
    bin_width = 22.5
    range_max = range_min + bin_number * bin_width
    myRange = [range_min,range_max]
    rng = np.random.default_rng(48)

    discard_percent_array = []    
    mean_difference_array= []
    sigma_difference_array = [] 
    for photon_num in photon_num_array:

        samples = rng.normal(normal_mean, normal_sigma, photon_num)

        pairs = [(float(p), 2) for p in samples]
        input_sample = [[pairs]]

        dead_time_samples,discard_count,accept_count = dead_time_simulation(simulated_photon=input_sample,imageWidth=1,imageHeight=1,pluse_train_num = pluse_train_num)
        discard_percent_array.append(1 - discard_count/photon_num)

        dead_time_distance = np.array([float(v) for v,_ in dead_time_samples[0][0]], dtype=float)
        dead_time_mean = float(dead_time_distance.mean())
        dead_time_var  = float(dead_time_distance.var(ddof=1))   # sample variance; use ddof=0 for population
        dead_time_sigma = np.sqrt(dead_time_var)
        mean_difference = normal_mean - dead_time_mean
        sigma_difference = normal_sigma - dead_time_sigma 
        mean_difference_array.append(mean_difference)
        sigma_difference_array.append(sigma_difference)

        print(mean_difference)
        print(sigma_difference)
        print()


    return discard_percent_array, mean_difference_array, sigma_difference_array 



input_photon_num = [(i+1)*100 + 1000 for i in range(200)]

T = 50*22.5e-12
rates_photons_per_s = [n / T for n in input_photon_num]


discard_ratio_1, mean_diff_1, sigma_diff_1 = dead_time_analysis(
    normal_mean=2436, normal_sigma=178, pluse_train_num=10000, photon_num_array=input_photon_num
)

discard_ratio_2, mean_diff_2, sigma_diff_2 = dead_time_analysis(
    normal_mean=2436, normal_sigma=178, pluse_train_num=20000, photon_num_array=input_photon_num
)

discard_ratio_3, mean_diff_3, sigma_diff_3 = dead_time_analysis(
    normal_mean=2436, normal_sigma=178, pluse_train_num=30000, photon_num_array=input_photon_num
)

discard_ratio_4, mean_diff_4, sigma_diff_4 = dead_time_analysis(
    normal_mean=2436, normal_sigma=178, pluse_train_num=40000, photon_num_array=input_photon_num
)

discard_ratio_5, mean_diff_5, sigma_diff_5 = dead_time_analysis(
    normal_mean=2436, normal_sigma=178, pluse_train_num=50000, photon_num_array=input_photon_num
)
plt.figure()
plt.plot(rates_photons_per_s, discard_ratio_1, label=f"{10000} pulse trains / measurement")
plt.plot(rates_photons_per_s, discard_ratio_2, label=f"{20000} pulse trains / measurement")
plt.plot(rates_photons_per_s, discard_ratio_3, label=f"{30000} pulse trains / measurement")
plt.plot(rates_photons_per_s, discard_ratio_4, label=f"{40000} pulse trains / measurement")
plt.plot(rates_photons_per_s, discard_ratio_5, label=f"{50000} pulse trains / measurement")
plt.xlabel("Photon flux (photons/s)")
plt.ylabel("Percentage of photon after dead time")
plt.legend(title="Pulse trains per measurement", loc="best")
plt.show()


# ---- 画图：双纵轴，共享同一 x ----
fig, ax1 = plt.subplots(figsize=(7.5, 4.5))

# 左轴：mean difference
l1, = ax1.plot(rates_photons_per_s, mean_diff_1, linewidth=1.6, color ="red", label="Mean difference")
ax1.set_xlabel("Photon flux")
ax1.set_ylabel("Mean difference")
# ax1.set_ylim(0,200)

# 右轴：sigma difference
ax2 = ax1.twinx()
l2, = ax2.plot(rates_photons_per_s, sigma_diff_1, linestyle="--", linewidth=1.6, label="Sigma difference")
ax2.set_ylabel("Sigma difference")

# 合并图例
lines = [l1, l2]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc="best")

# 网格与外观
ax1.grid(True, which="both", axis="both", linewidth=0.6, alpha=0.6)

plt.title("Dead-time analysis: mean vs sigma difference")
plt.tight_layout()
plt.show()

In [ ]:
%load_ext autoreload
%autoreload 2
from pythonLib.pixelationLib import*
from pythonLib.formImageLib import*
from scipy.stats import norm
from scipy.optimize import curve_fit

calibration_input_file_1 = "../ADS_calibration/pixelizedData/static_pixelized_3.h5"
calibration_input_file_2 = "../ADS_calibration/pixelizedData/static_pixelized_4.h5"
calibration_input_file_3 = "../ADS_calibration/pixelizedData/static_pixelized_5.h5"
calibration_input_file_4 = "../ADS_calibration/pixelizedData/static_pixelized_6.h5"



range_min = 1480
bin_number = 600
bin_width = 22.5
range_max = range_min + bin_number * bin_width
myRange = [range_min,range_max]

pixels_1, image_width, image_heigh = decode_file(calibration_input_file_1)
pixels_2, image_width, image_heigh = decode_file(calibration_input_file_2)
pixels_3, image_width, image_heigh = decode_file(calibration_input_file_3)
pixels_4, image_width, image_heigh = decode_file(calibration_input_file_4)


dead_time_pixels_1,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_1,imageWidth=image_width,imageHeight=image_heigh, pluse_train_num = 3500)
dead_time_pixels_2,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_2,imageWidth=image_width,imageHeight=image_heigh, pluse_train_num = 3500)
dead_time_pixels_3,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_3,imageWidth=image_width,imageHeight=image_heigh, pluse_train_num = 3500)
dead_time_pixels_4,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_4,imageWidth=image_width,imageHeight=image_heigh, pluse_train_num = 3500)



image_1, illegal_photon, stamped_histogram_1, stamped_collosion = form_histogram_image(pixels_1,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image_2, illegal_photon, stamped_histogram_2, stamped_collosion = form_histogram_image(pixels_2,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image_3, illegal_photon, stamped_histogram_3, stamped_collosion = form_histogram_image(pixels_3,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image_4, illegal_photon, stamped_histogram_4, stamped_collosion = form_histogram_image(pixels_4,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)



dead_time_image_1, illegal_photon, dead_time_stamped_histogram_1, dead_time_stamped_collosion = form_histogram_image(dead_time_pixels_1,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
dead_time_image_2, illegal_photon, dead_time_stamped_histogram_2, dead_time_stamped_collosion = form_histogram_image(dead_time_pixels_2,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
dead_time_image_3, illegal_photon, dead_time_stamped_histogram_3, dead_time_stamped_collosion = form_histogram_image(dead_time_pixels_3,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
dead_time_image_4, illegal_photon, dead_time_stamped_histogram_4, dead_time_stamped_collosion = form_histogram_image(dead_time_pixels_4,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)


histogram_120=display_pixel_histogram_from_matrix(stamped_histogram_1,25,25)
histogram_240=display_pixel_histogram_from_matrix(stamped_histogram_2,25,25)
histogram_360=display_pixel_histogram_from_matrix(stamped_histogram_3,25,25)
histogram_480=display_pixel_histogram_from_matrix(stamped_histogram_4,25,25)

dead_time_histogram_120=display_pixel_histogram_from_matrix(dead_time_stamped_histogram_1,25,25)
dead_time_histogram_240=display_pixel_histogram_from_matrix(dead_time_stamped_histogram_2,25,25)
dead_time_histogram_360=display_pixel_histogram_from_matrix(dead_time_stamped_histogram_3,25,25)
dead_time_histogram_480=display_pixel_histogram_from_matrix(dead_time_stamped_histogram_4,25,25)


# Fit 120 cm
mean_120, var_120 = estimate_normal_from_hist(histogram_120)
sigma_120 = np.sqrt(var_120)
x = np.linspace(0, len(histogram_120)-1, 600)
pdf_120 = norm.pdf(x, mean_120, sigma_120)
pdf_120_scaled = pdf_120 * np.max(histogram_120) / np.max(pdf_120)

# Fit 240 cm
mean_240, var_240 = estimate_normal_from_hist(histogram_240)
sigma_240 = np.sqrt(var_240)
pdf_240 = norm.pdf(x, mean_240, sigma_240)
pdf_240_scaled = pdf_240 * np.max(histogram_240) / np.max(pdf_240)

# Fit 360 cm
mean_360, var_360 = estimate_normal_from_hist(histogram_360)
sigma_360 = np.sqrt(var_360)
pdf_360 = norm.pdf(x, mean_360, sigma_360)
pdf_360_scaled = pdf_360 * np.max(histogram_360) / np.max(pdf_360)

# Fit 480 cm
mean_480, var_480 = estimate_normal_from_hist(histogram_480)
sigma_480 = np.sqrt(var_480)
pdf_480 = norm.pdf(x, mean_480, sigma_480)
pdf_480_scaled = pdf_480 * np.max(histogram_480) / np.max(pdf_480)

# Store results
x_data = np.array([(mean_120), (mean_240), (mean_360), (mean_480)])  # distances (cm)
y_data = np.array([np.max(histogram_120),np.max(histogram_240),np.max(histogram_360),np.max(histogram_480) ])  # mean bin

# --- Fit curve ---
popt, pcov = curve_fit(quad_func, x_data, y_data)
a, c = popt
print(f"Fitted coefficients:\n a = {a:.6e}\n c = {c:.6e}")

# --- Generate smooth fitted curve for plotting ---
x_fit = np.linspace(42, 600, 600)
y_fit = quad_func(x_fit, a, c)


bin_centers = np.arange(len(histogram_120))  # or use bin_width and range_min if needed
plt.figure()
plt.bar(bin_centers,histogram_120, width=1, alpha=0.6, color="blue", label="120cm")
plt.bar(bin_centers,histogram_240, width=1, alpha=0.6, color="red", label="240cm")
plt.bar(bin_centers,histogram_360, width=1, alpha=0.6, color="green", label="360cm")
plt.bar(bin_centers,histogram_480, width=1, alpha=0.6, color="orange", label="480cm")

plt.xlabel("Bin")
plt.ylabel("Count")

plt.plot(x, pdf_120_scaled, "k--", linewidth=1,
         label=f"Gaussian fit 120 (μ={(range_min+mean_120*bin_width)/20:.1f}, σ={sigma_120:.1f})")
plt.plot(x, pdf_240_scaled, "k--", linewidth=1,
         label=f"Gaussian fit 240 (μ={(range_min+mean_240*bin_width)/20:.1f}, σ={sigma_240:.1f})")
plt.plot(x, pdf_360_scaled, "k--", linewidth=1,
         label=f"Gaussian fit 360 (μ={(range_min+mean_360*bin_width)/20:.1f}, σ={sigma_360:.1f})")
plt.plot(x, pdf_480_scaled, "k--", linewidth=1,
         label=f"Gaussian fit 480 (μ={(range_min+mean_480*bin_width)/20:.1f}, σ={sigma_480:.1f})")

plt.plot(x_data, y_data, 'o',  color='red', markersize=3)
plt.plot(x_fit, y_fit, 'r--', linewidth=1)
plt.legend()
plt.grid()
plt.show()

# Fit 120 cm
dead_time_mean_120, dead_time_var_120 = estimate_normal_from_hist(dead_time_histogram_120)
dead_time_sigma_120 = np.sqrt(dead_time_var_120)
x = np.linspace(0, len(dead_time_histogram_120)-1, 600)
dead_time_pdf_120 = norm.pdf(x, dead_time_mean_120, dead_time_sigma_120)
dead_time_pdf_120_scaled = dead_time_pdf_120 * np.max(dead_time_histogram_120) / np.max(dead_time_pdf_120)

# Fit 240 cm
dead_time_mean_240, dead_time_var_240 = estimate_normal_from_hist(dead_time_histogram_240)
dead_time_sigma_240 = np.sqrt(dead_time_var_240)
dead_time_pdf_240 = norm.pdf(x, dead_time_mean_240, dead_time_sigma_240)
dead_time_pdf_240_scaled = dead_time_pdf_240 * np.max(dead_time_histogram_240) / np.max(dead_time_pdf_240)

# Fit 360 cm
dead_time_mean_360, dead_time_var_360 = estimate_normal_from_hist(dead_time_histogram_360)
dead_time_sigma_360 = np.sqrt(dead_time_var_360)
dead_time_pdf_360 = norm.pdf(x, dead_time_mean_360, dead_time_sigma_360)
dead_time_pdf_360_scaled = dead_time_pdf_360 * np.max(dead_time_histogram_360) / np.max(dead_time_pdf_360)

# Fit 480 cm
dead_time_mean_480, dead_time_var_480 = estimate_normal_from_hist(dead_time_histogram_480)
dead_time_sigma_480 = np.sqrt(dead_time_var_480)
dead_time_pdf_480 = norm.pdf(x, dead_time_mean_480, dead_time_sigma_480)
dead_time_pdf_480_scaled = dead_time_pdf_480 * np.max(dead_time_histogram_480) / np.max(dead_time_pdf_480)

# Store results
dead_time_x_data = np.array([(dead_time_mean_120), (dead_time_mean_240), (dead_time_mean_360), (dead_time_mean_480)])  # distances (cm)
dead_time_y_data = np.array([np.max(dead_time_histogram_120),np.max(dead_time_histogram_240),np.max(dead_time_histogram_360),np.max(dead_time_histogram_480) ])  # mean bin



plt.figure()
plt.bar(bin_centers,dead_time_histogram_120, width=1, alpha=0.6, color="blue", label="120cm")
plt.bar(bin_centers,dead_time_histogram_240, width=1, alpha=0.6, color="red", label="240cm")
plt.bar(bin_centers,dead_time_histogram_360, width=1, alpha=0.6, color="green", label="360cm")
plt.bar(bin_centers,dead_time_histogram_480, width=1, alpha=0.6, color="orange", label="480cm")
plt.xlabel("Bin")
plt.ylabel("Count")
plt.plot(x, dead_time_pdf_120_scaled, "k--", linewidth=1,
         label=f"Gaussian fit 120 (μ={(range_min+dead_time_mean_120*bin_width)/20:.1f}, σ={dead_time_sigma_120:.1f})")
plt.plot(x, dead_time_pdf_240_scaled, "k--", linewidth=1,
         label=f"Gaussian fit 240 (μ={(range_min+dead_time_mean_240*bin_width)/20:.1f}, σ={dead_time_sigma_240:.1f})")
plt.plot(x, dead_time_pdf_360_scaled, "k--", linewidth=1,
         label=f"Gaussian fit 360 (μ={(range_min+dead_time_mean_360*bin_width)/20:.1f}, σ={dead_time_sigma_360:.1f})")
plt.plot(x, dead_time_pdf_480_scaled, "k--", linewidth=1,
         label=f"Gaussian fit 480 (μ={(range_min+dead_time_mean_480*bin_width)/20:.1f}, σ={dead_time_sigma_480:.1f})")
plt.legend()
plt.grid()
plt.plot(dead_time_x_data, dead_time_y_data, 'o',  color='red', markersize=3)
plt.show()


In [ ]:
%load_ext autoreload
%autoreload 2
from pythonLib.pixelationLib import*
from pythonLib.formImageLib import*
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy.stats import poisson

calibration_input_file_1 = "../ADS_calibration/pixelizedData/static_pixelized_3.h5"
calibration_input_file_2 = "../ADS_calibration/pixelizedData/static_pixelized_4.h5"
calibration_input_file_3 = "../ADS_calibration/pixelizedData/static_pixelized_5.h5"
calibration_input_file_4 = "../ADS_calibration/pixelizedData/static_pixelized_6.h5"

range_min = 1480
bin_number = 600
bin_width = 22.5
range_max = range_min + bin_number * bin_width
myRange = [range_min,range_max]

pixels_1, image_width, image_heigh = decode_file(calibration_input_file_1)
pixels_2, image_width, image_heigh = decode_file(calibration_input_file_2)
pixels_3, image_width, image_heigh = decode_file(calibration_input_file_3)
pixels_4, image_width, image_heigh = decode_file(calibration_input_file_4)


# pixels_1,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_1,imageWidth=image_width,imageHeight=image_heigh, pluse_train_num = 1500)
# pixels_2,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_2,imageWidth=image_width,imageHeight=image_heigh, pluse_train_num = 1500)
# pixels_3,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_3,imageWidth=image_width,imageHeight=image_heigh, pluse_train_num = 1500)
# pixels_4,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels_4,imageWidth=image_width,imageHeight=image_heigh, pluse_train_num = 1500)

image_1, illegal_photon, stamped_histogram_1, stamped_collosion = form_histogram_image(pixels_1,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image_2, illegal_photon, stamped_histogram_2, stamped_collosion = form_histogram_image(pixels_2,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image_3, illegal_photon, stamped_histogram_3, stamped_collosion = form_histogram_image(pixels_3,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image_4, illegal_photon, stamped_histogram_4, stamped_collosion = form_histogram_image(pixels_4,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)

histogram_120=display_pixel_histogram_from_matrix(stamped_histogram_1,25,25)
histogram_240=display_pixel_histogram_from_matrix(stamped_histogram_2,25,25)
histogram_360=display_pixel_histogram_from_matrix(stamped_histogram_3,25,25)
histogram_480=display_pixel_histogram_from_matrix(stamped_histogram_4,25,25)



def estimate_poisson_lambda(pixel_hist):
    """MLE for Poisson over discrete bins k=0..N-1 given counts in each bin."""
    total = np.sum(pixel_hist)
    if total == 0:
        return np.nan
    k = np.arange(len(pixel_hist))
    lam = np.sum(k * pixel_hist) / total
    return lam

# --- Poisson fits ---
k = np.arange(len(histogram_120))  # discrete bin indices for PMF

# 120 cm
lam_120 = estimate_poisson_lambda(histogram_120)
pmf_120 = poisson.pmf(k, lam_120)
pmf_120_scaled = pmf_120 * np.max(histogram_120) / np.max(pmf_120)
sigma_120 = np.sqrt(lam_120)  # Poisson std

# 240 cm
lam_240 = estimate_poisson_lambda(histogram_240)
pmf_240 = poisson.pmf(k, lam_240)
pmf_240_scaled = pmf_240 * np.max(histogram_240) / np.max(pmf_240)
sigma_240 = np.sqrt(lam_240)

# 360 cm
lam_360 = estimate_poisson_lambda(histogram_360)
pmf_360 = poisson.pmf(k, lam_360)
pmf_360_scaled = pmf_360 * np.max(histogram_360) / np.max(pmf_360)
sigma_360 = np.sqrt(lam_360)

# 480 cm
lam_480 = estimate_poisson_lambda(histogram_480)
pmf_480 = poisson.pmf(k, lam_480)
pmf_480_scaled = pmf_480 * np.max(histogram_480) / np.max(pmf_480)
sigma_480 = np.sqrt(lam_480)

# --- Plot (unchanged bars; overlay Poisson instead of Gaussian) ---
bin_centers = np.arange(len(histogram_120))
plt.figure()
plt.bar(bin_centers, histogram_120, width=1, alpha=0.6, color="blue", label="120cm")
plt.bar(bin_centers, histogram_240, width=1, alpha=0.6, color="red", label="240cm")
plt.bar(bin_centers, histogram_360, width=1, alpha=0.6, color="green", label="360cm")
plt.bar(bin_centers, histogram_480, width=1, alpha=0.6, color="orange", label="480cm")
plt.xlabel("Bin")
plt.ylabel("Count")

# If you keep bin_width & range_min defined elsewhere, these labels convert bin index → distance like before
plt.plot(k, pmf_120_scaled, "k--", linewidth=1,
         label=f"Poisson 120 (λ={lam_120:.1f}, μ={(range_min+lam_120*bin_width)/20:.1f}, σ={sigma_120:.1f})")
plt.plot(k, pmf_240_scaled, "k--", linewidth=1,
         label=f"Poisson 240 (λ={lam_240:.1f}, μ={(range_min+lam_240*bin_width)/20:.1f}, σ={sigma_240:.1f})")
plt.plot(k, pmf_360_scaled, "k--", linewidth=1,
         label=f"Poisson 360 (λ={lam_360:.1f}, μ={(range_min+lam_360*bin_width)/20:.1f}, σ={sigma_360:.1f})")
plt.plot(k, pmf_480_scaled, "k--", linewidth=1,
         label=f"Poisson 480 (λ={lam_480:.1f}, μ={(range_min+lam_480*bin_width)/20:.1f}, σ={sigma_480:.1f})")

# plt.plot(x_data, y_data, 'o', color='red', markersize=3)  # your inverse-square fit points
# plt.plot(x_fit, y_fit, 'r--', linewidth=1)

plt.grid()
plt.legend()
plt.show()


In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw
import math


detector_size = 58
pixel_number = 50
fov = 45
d_distance = 1470


pixel_fov = fov / pixel_number
pixel_coverage = 2* ((math.tan(math.radians(pixel_fov/2)))* (d_distance + detector_size/2* math.cos(math.radians(pixel_fov/2)) / math.sin(math.radians(pixel_fov/2))))

print(pixel_coverage)


# --- your coordinates (image pixels; origin top-left) ---
W, H = 800, 1000
middle_point = (W/2, 10)

recive_point = middle_point 
project_point = (middle_point[0] - 20, middle_point[1])

A = (440, 880)
B = (360, 880)
# --------------------------------------------------------

img = Image.new("RGB", (W, H), "white")
draw = ImageDraw.Draw(img)

# draw line segment AB
draw.line((*A, *B), width=3, fill="black")

draw.line((*A, recive_point), width=3, fill="red")
draw.line((*A, project_point), width=3, fill="blue")

draw.line((*B, recive_point), width=3, fill="red")
draw.line((*B, project_point), width=3, fill="blue")
# draw the point
r = 5
draw.ellipse((project_point[0]-r, project_point[1]-r, project_point[0]+r, project_point[1]+r), fill="red")
draw.ellipse((recive_point[0]-r, recive_point[1]-r, recive_point[0]+r,recive_point[1]+r), fill="blue")
# optional: endpoints
for P, col in [(A, "green"), (B, "green")]:
    draw.ellipse((P[0]-4, P[1]-4, P[0]+4, P[1]+4), fill=col)

display(img)  # shows in notebook

def distance(p, q):
    """Euclidean distance between points p and q (iterables of equal length)."""
    if len(p) != len(q):
        raise ValueError("Points must have the same dimension.")
    return math.hypot(*((pi - qi) for pi, qi in zip(p, q)))


def sample_lambertian_2d_angle_truncated_interval(n_samples=1, a_deg=-1.9, b_deg=1.9,
                                                  rng=None, return_degrees=False):
    """
    从 2D 朗伯分布 p(θ)=0.5*cos(θ) 的截断区间 [a_deg, b_deg]（度）采样角度 θ。
    要求 -90° <= a_deg < b_deg <= 90°。
    反CDF：θ = asin( sin(a) + u * (sin(b) - sin(a)) ), u~U[0,1]，a,b为弧度。
    """
    if rng is None:
        rng = np.random.default_rng()
    if a_deg >= b_deg:
        a_deg, b_deg = b_deg, a_deg  # 容错：若传反则交换
    # 限制到物理支持区间（半圆）
    a_deg = max(a_deg, -90.0)
    b_deg = min(b_deg,  90.0)

    a = np.deg2rad(a_deg)
    b = np.deg2rad(b_deg)
    u = rng.random(n_samples)
    s = np.sin(a) + u * (np.sin(b) - np.sin(a))          # 线性插值到 [sin(a), sin(b)]
    theta_rad = np.arcsin(np.clip(s, -1.0, 1.0))         # 单调，arcsin返回[-π/2, π/2]

    if return_degrees:
        theta = np.rad2deg(theta_rad)
    else:
        theta = theta_rad
    return float(theta) if n_samples == 1 else theta

recive_point = (500+10+40, 10)
project_point = (500+40,10)

A = (500-50,1500)
B = (500+50,1500)
n = 5000  # number of points (inclusive)
x1, y1 = A
x2, y2 = B

points = [
    (x1 + (x2 - x1) * i / (n - 1),
     y1 + (y2 - y1) * i / (n - 1))
    for i in range(n)
]

ab_dir = (B[0] - A[0], B[1] - A[1])
angles_deg = sample_lambertian_2d_angle_truncated_interval(n_samples= len(points))
print([d_distance/np.cos(degree) for degree in angles_deg])
# Sum of distances to A and B for each point
distance_array = np.array([
    distance(p, project_point) + d_distance/ np.cos(deg)
    for p, deg in zip(points, angles_deg)
])


time_array = distance_array / 1000 / (3*10**8) * 10**12
# Plot
plt.figure()
plt.plot(range(len(distance_array)), distance_array, marker='o')
plt.xlabel("Point index along A→B (0..9)")
plt.ylabel("distance(point,A) + distance(point,B)")
plt.title("Sum of distances to A and B along the segment")
plt.grid(True)
plt.show()

print("Points:", points)
print("distance_array:", distance_array)
    

plt.figure()
plt.hist(time_array, bins='auto', edgecolor='black')
plt.xlabel("Distance sum")
plt.ylabel("Frequency")
plt.grid(True, axis='y', linestyle='--', alpha=0.5)
plt.show()




In [ ]:
%load_ext autoreload
%autoreload 2
from pythonLib.pixelationLib import*
from pythonLib.formImageLib import*



pixel_input_file = "../ADS_calibration/pixelizedData/static_pixelized_1.h5"
range_min = 1530
bin_number = 15
bin_width = 228
range_max = range_min + bin_number * bin_width
myRange = [range_min,range_max]

pixels, image_width, image_heigh = decode_file(pixel_input_file)
new_pixels,discard_count,accept_count = dead_time_simulation(simulated_photon=pixels,imageWidth=image_width,imageHeight=image_heigh)
print("discard_count : " + str(discard_count))

print("accept_count: " + str(accept_count))

pre_image, pre_illegal_photon, pre_stamped_histogram, pre_stamped_collosion = form_histogram_image(pixels,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
image, illegal_photon, stamped_histogram, stamped_collosion = form_histogram_image(new_pixels,image_width,image_heigh,bin_number=bin_number,range_distance=myRange)
display_image(image,distance_range=myRange)
display_image(pre_image,distance_range=myRange)
outputFile = "../dead_time_histogram.h5"
save_histogram_to_h5(outputFile, stamped_histogram, stamped_collosion ,range_min, range_max, image_width, image_heigh, bin_number)